In [1]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
%matplotlib inline
import seaborn as sns; sns.set_theme(color_codes=True)
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors
import warnings
warnings.filterwarnings('ignore')
from scipy import stats
import math    
import folium
from scipy.cluster.hierarchy import linkage, fcluster
from scipy.spatial.distance import pdist
import sys
from haversine import haversine
from shapely.geometry import LineString
from shapely.ops import split, linemerge
import krippendorff

In [2]:
from src.cluster import *
from src.colors import *
from src.map import *
from src.compare import *

In [3]:
#split line into segments of 30 ft
def split_line_at_intervals(line, interval=30):
    distance = 0
    segments = []

    while distance < line.length:
        sub_line = line.interpolate(distance)
        next_distance = distance + interval
        next_sub_line = line.interpolate(next_distance)
        segment = LineString([sub_line, next_sub_line])
        segments.append(segment)
        distance = next_distance

    return segments

def split_geodataframe(geodf, local_crs, interval=30):
    
    geodf = geodf.to_crs(local_crs)
    
    new_rows = []

    for index, row in geodf.iterrows():
        line = row.geometry
        splitted_lines = split_line_at_intervals(line, interval)

        for segment_id, segment in enumerate(splitted_lines):
            new_row = row.copy()
            new_row.geometry = segment
            new_row['segment_id'] = segment_id
            new_rows.append(new_row)

    df = gpd.GeoDataFrame(new_rows, columns=geodf.columns.append(pd.Index(['segment_id'])))
    #add a column that combines street_edge_id and segment_id
    df['street_edge_segment_id'] =  df['street_edge_id'].astype(str) + '_' +  df['segment_id'].astype(str)
    df.crs = local_crs
    return df

In [3]:
#read in seattle validation labels
seattle_labels = gpd.read_file('data/seattle-validation-labels-round2-230526.geojson')
seattle_labels = seattle_labels.drop(columns=['time_created'])
seattle_labels.head()

,username,label_id,street_edge_id,neighborhood,label_type,lat,lng,gsv_panorama_id,correct,severity,temporary,tag_list,description,geometry
0,Devon Snyder,4292,60,Ballard,NoCurbRamp,47.678276,-122.401039,ERmzzI8l1mh8UFmiubJraQ,None,5.0,f,no alternate route,None,POINT (-122.40104 47.67828)
1,Devon Snyder,4293,60,Ballard,NoCurbRamp,47.678257,-122.401031,ERmzzI8l1mh8UFmiubJraQ,None,5.0,f,no alternate route,None,POINT (-122.40103 47.67826)
2,Devon Snyder,4294,56,Ballard,NoCurbRamp,47.678185,-122.401009,ERmzzI8l1mh8UFmiubJraQ,None,5.0,f,no alternate route,None,POINT (-122.40101 47.67818)
3,Devon Snyder,4295,56,Ballard,NoCurbRamp,47.678165,-122.401031,ERmzzI8l1mh8UFmiubJraQ,None,5.0,f,no alternate route,None,POINT (-122.40103 47.67817)
4,Devon Snyder,4296,58,Ballard,NoCurbRamp,47.678291,-122.401169,ERmzzI8l1mh8UFmiubJraQ,None,5.0,f,no alternate route,None,POINT (-122.40117 47.67829)


In [4]:
seattle_labels[seattle_labels['label_type'] == 'Crosswalk']

,username,label_id,street_edge_id,neighborhood,label_type,lat,lng,gsv_panorama_id,correct,severity,temporary,tag_list,description,geometry
121,Devon Snyder,4415,107,Delridge,Crosswalk,47.537891,-122.361298,dnAl2N0kE6GOk6rv6U3GLg,None,1.0,f,None,None,POINT (-122.36130 47.53789)
126,Devon Snyder,4420,123,Delridge,Crosswalk,47.537827,-122.361137,dnAl2N0kE6GOk6rv6U3GLg,None,1.0,f,None,None,POINT (-122.36114 47.53783)
130,Devon Snyder,4424,122,Delridge,Crosswalk,47.538002,-122.361153,dnAl2N0kE6GOk6rv6U3GLg,None,1.0,f,None,None,POINT (-122.36115 47.53800)
133,Devon Snyder,4427,122,Delridge,Crosswalk,47.537945,-122.360886,DP0739Mjuw27VJ8IlzIwNg,None,1.0,f,None,None,POINT (-122.36089 47.53794)
157,Devon Snyder,4451,128,Delridge,Crosswalk,47.542801,-122.361214,TZyfamN5KbIquRE5NEfcbA,None,1.0,f,None,None,POINT (-122.36121 47.54280)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2924,mikey,7720,754,Windermere,Crosswalk,47.663002,-122.274704,mTmMuOg-GZQn9CQeSnfszA,None,1.0,f,None,None,POINT (-122.27470 47.66300)
2942,mikey,7738,786,Windermere,Crosswalk,47.664886,-122.274109,Sj44qqoLwi3d9_U7DSXRog,None,1.0,f,None,None,POINT (-122.27411 47.66489)
3004,mikey,7803,806,Windermere,Crosswalk,47.668163,-122.274124,5Yd9oX9Z5yqdVwORY1qQCA,None,1.0,f,None,None,POINT (-122.27412 47.66816)
3009,mikey,7808,706,Windermere,Crosswalk,47.668308,-122.274040,W6nkGfb2HZ6G7WhCELoh1Q,None,2.0,f,None,None,POINT (-122.27404 47.66831)


In [5]:
#read in seattle streets
seattle_streets = gpd.read_file("data/seattle-streets-230526.geojson")

In [6]:
#obtain cluster to street segment mapping
def cluster_to_street_segment_mapping(df,streets_seg):
     # Group by cluster_id and combine all geometries in each group into a MultiPoint
     df_grouped = df.groupby('cluster_id')['geometry'].apply(lambda x: x.unary_union)
     # Calculate centroid of each group
     centroids = df_grouped.centroid
     # Convert centroids to GeoDataFrame
     cluster_centroids = gpd.GeoDataFrame(geometry=centroids).reset_index()
     cluster_centroids.crs = {'init' :'epsg:4326'}
     cluster_centroids_ft = cluster_centroids.to_crs(epsg=streets_seg.crs.to_epsg())
     #map gdf_centroids to seattle_split, delete index_right column
     cluster_centeroids_street = cluster_centroids_ft.sjoin_nearest(streets_seg, distance_col='distance')
     cluster_centeroids_street = cluster_centeroids_street.drop(columns=['index_right'])
     #select only cluster_id and street_edge_segment_id
     cluster_centeroids_street = cluster_centeroids_street[['cluster_id','street_edge_segment_id']]
     #if there are multiple street_edge_segment_ids for a cluster_id, select the first one
     cluster_centeroids_street = cluster_centeroids_street.groupby('cluster_id').first().reset_index()
     return cluster_centeroids_street

In [7]:
#construct agreement table
def irr_table (df,street_seg):
    mikey = df[df['username'] == 'mikey']
    devon = df[df['username'] == 'Devon Snyder']
    irr_mikey = mikey.merge(street_seg, on='street_edge_segment_id', how='right').fillna(0)
    irr_devon = devon.merge(street_seg, on='street_edge_segment_id', how='right').fillna(0)
    #replace usernames with 1
    irr_mikey  = irr_mikey .replace('mikey',1)
    irr_devon  = irr_devon .replace('Devon Snyder',1)
    #group by street_edge_segment_id and count the number of label_ids
    irr_mikey = irr_mikey.groupby('street_edge_segment_id').sum()
    irr_devon = irr_devon.groupby('street_edge_segment_id').sum()
    irr_mikey = irr_mikey[['username']].reset_index()
    irr_devon = irr_devon[['username']].reset_index()
    irr_mikey = irr_mikey.rename(columns={'username':'mikey'})
    irr_devon = irr_devon.rename(columns={'username':'devon'})
    irr_mikey_devon = irr_mikey.merge(irr_devon, on='street_edge_segment_id', how='left').fillna(0)
    irr_mikey_devon = irr_mikey_devon.drop(columns=['street_edge_segment_id'])
    
    return irr_mikey_devon

In [8]:
def krippendorff_alpha(labels,label_type_id, streets, local_crs):
    #clustering by label type
    label_type = cluster_label_type_at_index(labels,label_type_id)[2]
    #split streets into segments
    street_seg = split_geodataframe(streets, local_crs, interval=30)
    #map cluster_id to street_edge_segment_id
    mapping = cluster_to_street_segment_mapping(label_type,street_seg)
    #update the street_edge_segment_id with the one closest to the cluster centroid
    label_type = label_type.merge(mapping, on='cluster_id', how='left')
    #select only the columns we need
    df = label_type[['username','cluster_id','street_edge_segment_id','label_id']]
    #construct agreement table
    irr = irr_table(df,street_seg)
    data_matrix = irr.T.to_numpy()
    alpha = krippendorff.alpha(data_matrix,level_of_measurement='ratio')

    return alpha.round(3)
    

In [9]:
alpha_seattle = {}
for i in [0,1,2,4,5,7,8,9]:
    alpha = krippendorff_alpha(seattle_labels,i, seattle_streets, 2285)
    alpha_seattle[label_types[i]] = alpha

In [12]:
#0607
alpha_seattle

{'CurbRamp': 0.938,
 'NoSidewalk': 0.878,
 'Problem': 0.525,
 'SurfaceProblem': 0.473,
 'Obstacle': 0.529,
 'NoCurbRamp': 0.74,
 'Crosswalk': 0.945,
 'Signal': 0.529}

In [11]:
for i in [0,1,2,4,5,7,8,9]:
    labels= map_clusters(seattle_labels,i)
    map = seattle_streets.explore(m = labels, tiles="CartoDb DarkMatter", color="#BF616A", style_kwds={"stroke":True,"opacity":0.4,"weight": 2},tooltip=["street_edge_id"], popup=["street_edge_id"])
    map.save("html/seattle_"+label_types[i]+".html")


In [13]:
#take out label type = surfaceproblem and obstacle
seattle_problems= seattle_labels[(seattle_labels['label_type'] == 'SurfaceProblem') | (seattle_labels['label_type'] == 'Obstacle') ]
#take out severity ==1
seattle_problems = seattle_problems[seattle_problems['severity'] != 1]
seattle_problems

,username,label_id,street_edge_id,neighborhood,label_type,lat,lng,gsv_panorama_id,correct,severity,temporary,tag_list,description,geometry
37,Devon Snyder,4330,39,Ballard,SurfaceProblem,47.677654,-122.401009,1I5HEo2VLaST2hYoiIbMCA,None,2.0,f,"cracks,grass",None,POINT (-122.40101 47.67765)
42,Devon Snyder,4335,39,Ballard,SurfaceProblem,47.677494,-122.400986,11eaYUb08jpmHzB35OXojw,None,2.0,f,None,None,POINT (-122.40099 47.67749)
67,Devon Snyder,4360,494,Ballard,SurfaceProblem,47.676315,-122.400986,XBIw1Ra9ctGK6i51cpnblA,None,2.0,f,cracks,None,POINT (-122.40099 47.67632)
107,Devon Snyder,4401,104,Delridge,SurfaceProblem,47.537506,-122.362396,7rtGfmwUxo5KmMttHSV9Ww,None,4.0,f,"cracks,very broken",None,POINT (-122.36240 47.53751)
115,Devon Snyder,4409,104,Delridge,SurfaceProblem,47.537910,-122.361740,vYNn_2-TwKdQR0nYjd7cnA,None,2.0,f,cracks,None,POINT (-122.36174 47.53791)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3039,mikey,7839,792,Windermere,SurfaceProblem,47.661369,-122.273071,vuv99kGmxqvcoeJSgwnA-A,None,3.0,f,height difference,None,POINT (-122.27307 47.66137)
3041,mikey,7841,792,Windermere,SurfaceProblem,47.661373,-122.273483,MAdphQuzuJUm0QncUopjeQ,None,3.0,f,height difference,None,POINT (-122.27348 47.66137)
3052,mikey,7852,809,Windermere,SurfaceProblem,47.661343,-122.271935,eVtqqMNfwUZvuibQQ5N6LA,None,2.0,f,None,None,POINT (-122.27193 47.66134)
3061,mikey,7861,809,Windermere,SurfaceProblem,47.661247,-122.271759,Kv4J_ph-JeOR1prz5jKDug,None,2.0,f,cracks,None,POINT (-122.27176 47.66125)


In [14]:
alpha_seattle_problems = {}
for i in [2,4,5]:
    alpha = krippendorff_alpha(seattle_problems,i, seattle_streets, 3435)
    alpha_seattle_problems[label_types[i]] = alpha

In [15]:
alpha_seattle_problems

{'Problem': 0.505, 'SurfaceProblem': 0.483, 'Obstacle': 0.541}

In [16]:
for i in [2,4,5]:
    labels= map_clusters(seattle_problems,i)
    map = seattle_streets.explore(m = labels, tiles="CartoDb DarkMatter", color="#BF616A", style_kwds={"stroke":True,"opacity":0.4,"weight": 2},tooltip=["street_edge_id"])
    map.save("html/seattle_wo_sev1_"+label_types[i]+".html")